UserCF:基于用户的协同过滤算法。<br>
UserIIF:改进UserCF中用户相似度的计算。<br>
数据来源：movieLens(train.csv训练集 + test.csv测试集) <br>

In [55]:
import os
import yaml
import dask.dataframe as dd
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
from operator import itemgetter
%matplotlib inline

In [40]:
data = pd.read_csv('train.csv')

In [72]:
## 1.读取训练集
train = dict()  # 训练集
csvFileName = r'train.csv'
with open(csvFileName,'r',encoding='UTF-8') as file:
    rows = csv.reader(file)
    for row in rows:
        if row[3] not in train.keys():
            train[row[3]] = {row[0]:row[4]}
        else:
            train[row[3]][row[0]] = row[4]
train.pop('userId')
#train


{'movieId': 'rating'}

In [1]:
test_data = {'A':{'a':3,'b':4,'d':5},
            'B':{'a':4,'c':4.5},
            'C':{'b':4.5,'e':4},
            'D':{'c':3.5,'d':4,'e':2.5}}

In [35]:
## train = {'userId':{'movieId':rating}}
def UserSimilarity(train):
    """根据余弦计算用户相似度"""
    item_users = dict()
    ## item_user= {'movieId':(userId1,userId2..),...}  建立电影：（用户）的反向表
    for u,items in train.items():
        for i in items.keys():
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)
   # print("item_users:",item_users)
    
    # calculate co-ralated items between users
    C = dict()
    N = dict()
    for movie,users in item_users.items():
        for u in users:
            if u not in N.keys():
                N[u] = 0
            N[u] += 1
            for v in users:
                if u == v: continue
                if u not in C.keys():
                    C[u] = {v:0}
                elif v not in C[u].keys():
                    C[u][v] = 0
                C[u][v] += 1
   # print("C:",C)         
    
    # calculate simularity matrix W
    W = dict()
    for u, related_users in C.items():
        for v,cuv in related_users.items():
            if u not in W.keys():
                W[u] = {v:0}
            W[u][v] = cuv/math.sqrt(N[u]*N[v])
    # print("W:",W)
    return W
    

In [91]:
def UserSimilarity_UserIIF(train):
    """改进UserCF。UserIIF:惩罚用户u和用户v共同兴趣列表中热门物品的影响"""
    item_users = dict()
    ## item_user= {'movieId':(userId1,userId2..),...}  建立电影：（用户）的反向表
    for u,items in train.items():
        for i in items.keys():
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)
   
    # calculate co-ralated items between users
    C = dict()
    N = dict()
    for movie,users in item_users.items():
        for u in users:
            if u not in N.keys():
                N[u] = 0
            N[u] += 1
            for v in users:
                if u == v: continue
                if u not in C.keys():
                    C[u] = {v:0}
                elif v not in C[u].keys():
                    C[u][v] = 0
                C[u][v] += 1/math.log(1+len(users))
   # print("C:",C)         
    
    # calculate simularity matrix W
    W = dict()
    for u, related_users in C.items():
        for v,cuv in related_users.items():
            if u not in W.keys():
                W[u] = {v:0}
            W[u][v] = cuv/math.sqrt(N[u]*N[v])
    # print("W:",W)
    return W

In [92]:
'''
W = UserSimilarity(test_data)
print("W:",W)
'''
W = UserSimilarity_UserIIF(test_data)
W

{'B': {'A': 0.37160360818355515, 'D': 0.37160360818355515},
 'A': {'B': 0.37160360818355515,
  'C': 0.37160360818355515,
  'D': 0.3034130755422791},
 'C': {'A': 0.37160360818355515, 'D': 0.37160360818355515},
 'D': {'A': 0.3034130755422791,
  'B': 0.37160360818355515,
  'C': 0.37160360818355515}}

In [93]:
def recommend(user,train,W,K):
    rank = dict()
    interacted_items = train[user]  # 用户user看过的电影
    for v,wuv in sorted(W[user].items(),key = lambda d:d[1],reverse=True)[0:K]:
        #  v:用户v  wuv:用户u对v的相似度
        #print("user_v:%s wuv:%s"%(v,wuv))
        for i,rvi in train[v].items():
            #print("movie_i:%s rvi:%s"%(i,rvi))
            # i:movie  rvi:用户v对电影i的评分
            if i in interacted_items:continue
            if i not in rank.keys():
                rank[i] = 0
            rank[i] += wuv*rvi
    return rank

In [94]:
rank = recommend('A',test_data,W,3)
rank

{'c': 2.734162001223975, 'e': 2.2449471215899184}